In [42]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# CONSTS

In [43]:
CHROME_DRIVER_PATH = "./chromedriver"
BRAVE_PATH = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
DOWNLOAD_DIRECTORY = "/Users/mamoudou/Developer/Python/ProjetTrutoré/"
ARCHIVE_DIRECTORY = "/Users/mamoudou/Developer/Python/ProjetTrutoré/archive"
URL = "https://donneespubliques.meteofrance.fr/?fond=produit&id_produit=90&id_rubrique=32"

## Browser Configs

In [44]:
options = Options()
options.binary_location = BRAVE_PATH  

prefs = {
    "download.default_directory": DOWNLOAD_DIRECTORY,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True 
}
options.add_experimental_option("prefs", prefs)

In [45]:
service = Service(executable_path=CHROME_DRIVER_PATH)

driver = webdriver.Chrome(service=service, options=options)



driver.get(URL)
print("Page Title:", driver.title)
telecharger_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Télécharger']")
telecharger_button.click()

time.sleep(5)
driver.quit()

Page Title: Données Publiques de Météo-France - Données SYNOP essentielles OMM


## Fixing the file extension problem

In [46]:
from file_management import change_file_extension

cvs_file = change_file_extension(DOWNLOAD_DIRECTORY)

Le fichier a été renommé : /Users/mamoudou/Developer/Python/ProjetTrutoré/synop.2024122621.txt -> /Users/mamoudou/Developer/Python/ProjetTrutoré/synop.2024122621.csv


# Starting the ELT

In [47]:
import pandas as pd
import numpy as np
df = pd.read_csv(cvs_file, sep=";", dtype={"date": str})

print(df.to_string())

    numer_sta            date    pmer  tend cod_tend   dd    ff           t          td    u     vv  ww  w1  w2    n nbas  hbas  cl  cm  ch    pres niv_bar geop tend24        tn12        tn24        tx12        tx24     tminsol  sw  tw      raf10     rafper  per etat_sol  ht_neige    ssfrai perssfrai        rr1        rr3        rr6       rr12       rr24 phenspe1 phenspe2 phenspe3 phenspe4 nnuage1 ctype1 hnuage1 nnuage2 ctype2 hnuage2 nnuage3 ctype3 hnuage3 nnuage4 ctype4 hnuage4  Unnamed: 59
0        7005  20241226210000  103490    10        0   70   2.1  277.150000  277.150000  100    300  45  mq  mq   mq   mq    mq  mq  mq  mq  102580      mq   mq     mq          mq          mq          mq          mq          mq  mq  mq         mq   4.100000  -10       mq  0.000000        mq        mq   0.000000   0.000000         mq         mq         mq       mq       mq       mq       mq      mq     mq      mq      mq     mq      mq      mq     mq      mq      mq     mq      mq          NaN
1   

# Récupération des informations importantes à savoir :
1. Indicatif OMM station : numéro de station `numer_sta` -> `int`
2. Date (UTC) : `date` -> `datetime`
3. Pression au niveau mer : `pmer` -> `int`
4. Variation de pression en 3 heures : `tend` -> `int`
5. Type de tendance barométrique : `cod_tend` -> `int`
6. Direction du vent moyen 10 mn : `dd` -> `int`
7. Vitesse du vent moyen 10 mn : `ff` -> `float`
8. Température : `t` -> `float`
9. Point de rosée : `td` -> `float`
10. Humidité : `u` -> `int`
11. Visibilité horizontale : `vv` -> `float`
12. Temps présent : `ww` -> `int`
13. Nébulosité totale : `n` -> `float`
14. Nébulosité des nuages de l'étage inférieur : `nbas` -> `int`
15. Hauteur de la base des nuages de l'étage inférieur : `hbas` -> `int`
16. Pression station : `pres` -> `int`
17. Variation de pression en 24 heures : `tend24` -> `int`
18. Température minimale sur N heures : `tn12` -> `float`
19. Température maximale sur N heures : `tx12` -> `float`
20. Température minimale du sol sur 12 heures : `tminsol` -> `float`
21. Rafales sur les 10 dernières minutes : `raf10` -> `float`
22. Rafales sur une période : `rafper` -> `float`
23. Période de mesure des rafales : `per` -> `float`
24. Précipitations dans les N dernières heures : `rr12` -> `float`

In [48]:
df = df[['numer_sta', 'date', "pmer", "tend", "cod_tend", "dd", "ff", "t", "td", "u", "vv", "ww", "n", "nbas", "hbas","pres", "tend24", "tn12", "tx12", "tminsol", "raf10", "rafper", "per", "rr12" ]]


print(df.to_string())


    numer_sta            date    pmer  tend cod_tend   dd    ff           t          td    u     vv  ww    n nbas  hbas    pres tend24        tn12        tx12     tminsol      raf10     rafper  per       rr12
0        7005  20241226210000  103490    10        0   70   2.1  277.150000  277.150000  100    300  45   mq   mq    mq  102580     mq          mq          mq          mq         mq   4.100000  -10         mq
1        7015  20241226210000  103530     0        0   60   3.0  276.950000  276.950000  100    140  45  101    9    30  102930    -30          mq          mq  274.850000   4.500000   5.000000  -10   0.000000
2        7020  20241226210000  103310   -60        8   80   6.0  282.050000  282.050000  100     mq  mq   mq   mq    mq  103200   -150          mq          mq          mq   7.900000   8.800000  -10   0.000000
3        7027  20241226210000  103370   -10        8   90   2.8  279.850000  279.750000   99    210  45  101    9    30  102530   -180          mq          mq  281.

## Remplacer les valeurs manquantes `mq` par `NaN`

In [49]:
df.replace("mq", np.nan, inplace=True)
print(df.to_string())

    numer_sta            date    pmer  tend cod_tend   dd    ff           t          td    u     vv   ww    n nbas  hbas    pres tend24        tn12        tx12     tminsol      raf10     rafper  per       rr12
0        7005  20241226210000  103490    10        0   70   2.1  277.150000  277.150000  100    300   45  NaN  NaN   NaN  102580    NaN         NaN         NaN         NaN        NaN   4.100000  -10        NaN
1        7015  20241226210000  103530     0        0   60   3.0  276.950000  276.950000  100    140   45  101    9    30  102930    -30         NaN         NaN  274.850000   4.500000   5.000000  -10   0.000000
2        7020  20241226210000  103310   -60        8   80   6.0  282.050000  282.050000  100    NaN  NaN  NaN  NaN   NaN  103200   -150         NaN         NaN         NaN   7.900000   8.800000  -10   0.000000
3        7027  20241226210000  103370   -10        8   90   2.8  279.850000  279.750000   99    210   45  101    9    30  102530   -180         NaN         NaN 

## Nettoyages de colonnes non renseignées 

1. Informations générales
	-	Numéro de station : `numer_sta` -> int
	-	Date (UTC) : `date` -> datetime
2. Pression
	-	Pression au niveau mer : `pmer` -> int
	-	Variation de pression en 3 heures : `tend` -> int
	-	Type de tendance barométrique : `cod_tend` -> int
	-	Pression station : `pres` -> int
	-	Variation de pression en 24 heures : `tend24` -> int
3. Vent
	-	Direction du vent moyen 10 mn : `dd` -> int
	-	Vitesse du vent moyen 10 mn : `ff` -> float
	-	Rafales sur les 10 dernières minutes : `raf10` -> float
	-	Rafales sur une période : `rafper` -> float
	-	Période de mesure des rafales : `per` -> float
4. Température
	-	Température : `t` -> float
	-	Point de rosée : `td` -> float
	-	Température minimale sur N heures : `tn12` -> float
	-	Température maximale sur N heures : `tx12` -> float
	-	Température minimale du sol sur 12 heures : `tminsol` -> float
5. Humidité
	-	Humidité : `u` -> int
6. Visibilité
	-	Visibilité horizontale : `vv` -> float
7. Nébulosité et nuages
	-	Nébulosité totale : `n` -> float
	-	Nébulosité des nuages de l’étage inférieur : `nbas` -> int
	-	Hauteur de la base des nuages de l’étage inférieur : `hbas` -> int
8. Précipitations
	-	Précipitations dans les N dernières heures : `rr12` -> float
9. Temps présent
	-	Temps présent : `ww` -> int

In [50]:
df[['dd', 'ff', 'raf10', 'rafper', 'rr12', "per"]] = df[['dd', 'ff', 'raf10', 'rafper', 'rr12', "per"]].fillna(0)

moyenne = ["pmer", "tend", "cod_tend", "pres", "tend24", "t", "td", "tn12", "tx12", "tminsol", "u", "vv", "n", "nbas", "hbas", "ww"]
df[moyenne] = df[moyenne].apply(pd.to_numeric, errors='coerce')

for col in moyenne:
    df[col] = df[col].fillna(df[col].mean())

print(df.to_string())

    numer_sta            date           pmer        tend  cod_tend   dd    ff          t          td          u            vv         ww       n   nbas         hbas           pres      tend24    tn12    tx12     tminsol      raf10     rafper  per       rr12
0        7005  20241226210000  103490.000000   10.000000  0.000000   70   2.1  277.15000  277.150000  100.00000    300.000000  45.000000   77.25  3.325   445.714286  102580.000000  -53.272727  269.35  275.65  277.426471          0   4.100000  -10          0
1        7015  20241226210000  103530.000000    0.000000  0.000000   60   3.0  276.95000  276.950000  100.00000    140.000000  45.000000  101.00  9.000    30.000000  102930.000000  -30.000000  269.35  275.65  274.850000   4.500000   5.000000  -10   0.000000
2        7020  20241226210000  103310.000000  -60.000000  8.000000   80   6.0  282.05000  282.050000  100.00000  17790.869565   9.822222   77.25  3.325   445.714286  103200.000000 -150.000000  269.35  275.65  277.426471   7.90

## Typages des colonnes

In [51]:
df["date"] = pd.to_datetime(df["date"], format='%Y%m%d%H%M%S')
df['date'] = df['date'].dt.strftime("%Y-%m-%d %H:%M:%S")

entier = ["numer_sta", "pmer", "tend", "cod_tend", "dd", "u", "ww", "nbas", "hbas", "pres", "tend24"]
for i in entier:
    print(f"{i} -> {df[col].dtype}")
    df[entier] = df[entier].astype(int)
    print(f"{i} -> {df[col].dtype}")
    print("")

double = ['dd', 'ff', 'raf10', 'rafper', 'rr12', "per", "t", "td", "tn12", "tx12", "tminsol", "n", "vv"]
for i in double:
    print(f"{i} -> {df[col].dtype}")
    df[double] = df[double].astype(float)
    print(f"{i} -> {df[col].dtype}")
    print("")

numer_sta -> float64
numer_sta -> int64

pmer -> int64
pmer -> int64

tend -> int64
tend -> int64

cod_tend -> int64
cod_tend -> int64

dd -> int64
dd -> int64

u -> int64
u -> int64

ww -> int64
ww -> int64

nbas -> int64
nbas -> int64

hbas -> int64
hbas -> int64

pres -> int64
pres -> int64

tend24 -> int64
tend24 -> int64

dd -> int64
dd -> int64

ff -> int64
ff -> int64

raf10 -> int64
raf10 -> int64

rafper -> int64
rafper -> int64

rr12 -> int64
rr12 -> int64

per -> int64
per -> int64

t -> int64
t -> int64

td -> int64
td -> int64

tn12 -> int64
tn12 -> int64

tx12 -> int64
tx12 -> int64

tminsol -> int64
tminsol -> int64

n -> int64
n -> int64

vv -> int64
vv -> int64



In [52]:
print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff          t          td    u            vv  ww       n  nbas  hbas    pres  tend24    tn12    tx12     tminsol  raf10  rafper   per  rr12
0        7005  2024-12-26 21:00:00  103490    10         0   70.0   2.1  277.15000  277.150000  100    300.000000  45   77.25     3   445  102580     -53  269.35  275.65  277.426471    0.0     4.1 -10.0   0.0
1        7015  2024-12-26 21:00:00  103530     0         0   60.0   3.0  276.95000  276.950000  100    140.000000  45  101.00     9    30  102930     -30  269.35  275.65  274.850000    4.5     5.0 -10.0   0.0
2        7020  2024-12-26 21:00:00  103310   -60         8   80.0   6.0  282.05000  282.050000  100  17790.869565   9   77.25     3   445  103200    -150  269.35  275.65  277.426471    7.9     8.8 -10.0   0.0
3        7027  2024-12-26 21:00:00  103370   -10         8   90.0   2.8  279.85000  279.750000   99    210.000000  45  101.00     9    30  102530    -180  269.35  2

## Conversion des températures de Kelvin en Celcuis

In [53]:
celcuis = ["t", "td", "tn12", "tx12", "tminsol"]
for i in celcuis:
    df[i] = df[i] - 273.15

print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff         t         td    u            vv  ww       n  nbas  hbas    pres  tend24  tn12  tx12    tminsol  raf10  rafper   per  rr12
0        7005  2024-12-26 21:00:00  103490    10         0   70.0   2.1   4.00000   4.000000  100    300.000000  45   77.25     3   445  102580     -53  -3.8   2.5   4.276471    0.0     4.1 -10.0   0.0
1        7015  2024-12-26 21:00:00  103530     0         0   60.0   3.0   3.80000   3.800000  100    140.000000  45  101.00     9    30  102930     -30  -3.8   2.5   1.700000    4.5     5.0 -10.0   0.0
2        7020  2024-12-26 21:00:00  103310   -60         8   80.0   6.0   8.90000   8.900000  100  17790.869565   9   77.25     3   445  103200    -150  -3.8   2.5   4.276471    7.9     8.8 -10.0   0.0
3        7027  2024-12-26 21:00:00  103370   -10         8   90.0   2.8   6.70000   6.600000   99    210.000000  45  101.00     9    30  102530    -180  -3.8   2.5   8.300000    3.8     4.2 -1

## Arrondir tous les decimaux à 2chiffres apes la virgules

In [54]:
df[double] = df[double].round(2)
print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff     t     td    u        vv  ww       n  nbas  hbas    pres  tend24  tn12  tx12  tminsol  raf10  rafper   per  rr12
0        7005  2024-12-26 21:00:00  103490    10         0   70.0   2.1   4.0   4.00  100    300.00  45   77.25     3   445  102580     -53  -3.8   2.5     4.28    0.0     4.1 -10.0   0.0
1        7015  2024-12-26 21:00:00  103530     0         0   60.0   3.0   3.8   3.80  100    140.00  45  101.00     9    30  102930     -30  -3.8   2.5     1.70    4.5     5.0 -10.0   0.0
2        7020  2024-12-26 21:00:00  103310   -60         8   80.0   6.0   8.9   8.90  100  17790.87   9   77.25     3   445  103200    -150  -3.8   2.5     4.28    7.9     8.8 -10.0   0.0
3        7027  2024-12-26 21:00:00  103370   -10         8   90.0   2.8   6.7   6.60   99    210.00  45  101.00     9    30  102530    -180  -3.8   2.5     8.30    3.8     4.2 -10.0   0.0
4        7037  2024-12-26 21:00:00  103430   -30         8  

# Archive file

In [55]:
from file_management import move_file

move_file(cvs_file, ARCHIVE_DIRECTORY)

Le fichier a été déplacé de /Users/mamoudou/Developer/Python/ProjetTrutoré/synop.2024122621.csv à /Users/mamoudou/Developer/Python/ProjetTrutoré/archive/synop.2024122621.csv


## Converting DF to JSON and then to readable dict

In [56]:
from datetime import datetime

In [57]:
import pprint   
import json

json_data = df.to_json(orient='records')
dict_data = json.loads(json_data)

pprint.pprint(dict_data)

[{'cod_tend': 0,
  'date': '2024-12-26 21:00:00',
  'dd': 70.0,
  'ff': 2.1,
  'hbas': 445,
  'n': 77.25,
  'nbas': 3,
  'numer_sta': 7005,
  'per': -10.0,
  'pmer': 103490,
  'pres': 102580,
  'raf10': 0.0,
  'rafper': 4.1,
  'rr12': 0.0,
  't': 4.0,
  'td': 4.0,
  'tend': 10,
  'tend24': -53,
  'tminsol': 4.28,
  'tn12': -3.8,
  'tx12': 2.5,
  'u': 100,
  'vv': 300.0,
  'ww': 45},
 {'cod_tend': 0,
  'date': '2024-12-26 21:00:00',
  'dd': 60.0,
  'ff': 3.0,
  'hbas': 30,
  'n': 101.0,
  'nbas': 9,
  'numer_sta': 7015,
  'per': -10.0,
  'pmer': 103530,
  'pres': 102930,
  'raf10': 4.5,
  'rafper': 5.0,
  'rr12': 0.0,
  't': 3.8,
  'td': 3.8,
  'tend': 0,
  'tend24': -30,
  'tminsol': 1.7,
  'tn12': -3.8,
  'tx12': 2.5,
  'u': 100,
  'vv': 140.0,
  'ww': 45},
 {'cod_tend': 8,
  'date': '2024-12-26 21:00:00',
  'dd': 80.0,
  'ff': 6.0,
  'hbas': 445,
  'n': 77.25,
  'nbas': 3,
  'numer_sta': 7020,
  'per': -10.0,
  'pmer': 103310,
  'pres': 103200,
  'raf10': 7.9,
  'rafper': 8.8,
  'rr1

In [58]:
first = dict_data[0]
js = json.dumps(first)

print(js)

{"numer_sta": 7005, "date": "2024-12-26 21:00:00", "pmer": 103490, "tend": 10, "cod_tend": 0, "dd": 70.0, "ff": 2.1, "t": 4.0, "td": 4.0, "u": 100, "vv": 300.0, "ww": 45, "n": 77.25, "nbas": 3, "hbas": 445, "pres": 102580, "tend24": -53, "tn12": -3.8, "tx12": 2.5, "tminsol": 4.28, "raf10": 0.0, "rafper": 4.1, "per": -10.0, "rr12": 0.0}
